## 1D TFIM

Working with the Transverse Field Ising Model (TFIM).
The Hamiltonian has the following form:

$H = -J\sum_{i=1}^{N-1} \sigma_{i}^{z}\sigma_{i+1}^{z} - J\sigma_{N}\sigma_{1} - h\sum_{i = 1}^{N}\sigma_{i}^{x}$

(Note: The sum is over nearest neighbors, which is written as i, i+1 since we are looking specifically at 1D case. Since we are working with PBC, we can simply add the boundary term.)

## Step 1: Build Hamiltonian

In [62]:
function Hamiltonian(N,J,h)
    
    #Dimension of the Hilbert Space
    dim = (2)^N
    #Initialize a matrix for the Hamiltonian
    H = zeros(dim,dim)
    
    #Loop over the basis kets to construct the Hamiltonian
    #Basically, you know which inner products will contribute based on the form of H
    for ket in (0:dim-1)
        
        #Put ket in binary (printing purposes only)
        ket_binary = bitstring(ket)
        #Initialize the diagonal element of the matrix
        Diagonal = Int64(0)
        
        #Loop over each spin in the ket ad manipulate it as needed to construct the diagonal and off-diagonal 
        #pieces of the Hamiltonian
        for SpinIndex in (0:N-1) 
            #get the binary representation of the bit you are looking at
            bit = Int(2)^(SpinIndex)
            #XOR with ket 
            #This flips the spin corresponding to SpinIndex 
            #This generates the bras that will contribute via the transverse field
            bra = ket ⊻ bit
            #include the transverse field contributions to the Hamiltonian matrix
            H[bra+1,ket+1] += h
            
            #For the last spin, calculate the periodic boundary term
            if SpinIndex == N-1
                S_last = 2*((ket>>SpinIndex)&1)-1
                S_first = 2*((ket>>(0))&1)-1
                Bond = J*S_last*S_first
                Diagonal += Bond
                break
            end
            
            #Calculate the diagonal term
            Si = 2*((ket>>SpinIndex)&1)-1
            Si_next = 2*((ket>>(SpinIndex+1))&1)-1
            Bond = J*Si*Si_next
            Diagonal += Bond
        end
        
        #Set the diagonal terms of the Hamiltonian
        H[ket+1,ket+1] += -1 * (Diagonal)
        
    end
    
    return H
    
end

Hamiltonian (generic function with 2 methods)

##### Test this function to make sure it returns the proper Hamiltonian.

In [60]:
using LinearAlgebra

n = 3
j = 1
h = 1
Ham = Hamiltonian(n,j,h)
eigs = eigvals(Ham)

println("The Hamiltonian for N = ",n,", J = ",j,",and h = ",h," is")
show(stdout,"text/plain",Ham)
println("")
println("with eigenvalues,")
println(eigs)



The Hamiltonian for N = 3, J = 1,and h = 1 is
8×8 Matrix{Float64}:
 -3.0  1.0  1.0  0.0  1.0  0.0  0.0   0.0
  1.0  1.0  0.0  1.0  0.0  1.0  0.0   0.0
  1.0  0.0  1.0  1.0  0.0  0.0  1.0   0.0
  0.0  1.0  1.0  1.0  0.0  0.0  0.0   1.0
  1.0  0.0  0.0  0.0  1.0  1.0  1.0   0.0
  0.0  1.0  0.0  0.0  1.0  1.0  0.0   1.0
  0.0  0.0  1.0  0.0  1.0  0.0  1.0   1.0
  0.0  0.0  0.0  1.0  0.0  1.0  1.0  -3.0
with eigenvalues,
[-4.000000000000001, -3.464101615137755, -3.2331942489481536e-16, 9.773829788553912e-17, 4.62097303717605e-16, 2.0, 2.0000000000000013, 3.464101615137753]


## Step 2: Diagonalize the Hamiltonian

In [78]:
using LinearAlgebra

n_ = 3
j_ = 1
h_ = 0
Ham = Hamiltonian(n_,j_,h_)

eigen_vals = eigvals(Ham)
eigen_vecs = eigvecs(Ham)

show(stdout,"text/plain",eigen_vals)
println("")
show(stdout,"text/plain",eigen_vecs)

S = eigen_vecs
D = diagm(eigen_vals)
S_inv = inv(S)

8-element Vector{Float64}:
 -3.0
 -3.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
8×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0

8×8 Matrix{Float64}:
 1.0  0.0  -0.0  0.0  -0.0  0.0  -0.0  -0.0
 0.0  0.0  -0.0  0.0  -0.0  0.0  -0.0   1.0
 0.0  1.0  -0.0  0.0  -0.0  0.0  -0.0   0.0
 0.0  0.0   1.0  0.0  -0.0  0.0  -0.0   0.0
 0.0  0.0   0.0  1.0  -0.0  0.0  -0.0   0.0
 0.0  0.0   0.0  0.0   1.0  0.0  -0.0   0.0
 0.0  0.0   0.0  0.0   0.0  1.0  -0.0   0.0
 0.0  0.0   0.0  0.0   0.0  0.0   1.0   0.0

##### Check this diagonalization (ie does it reproduce original Hamiltonian)

In [71]:
using LinearAlgebra

Ham_reproduced = lmul!(S,D)
Ham_reproduced = lmul!(Ham_reproduced,S_inv)

LoadError: MethodError: no method matching lmul!(::Matrix{Float64}, ::Matrix{Float64})
[0mClosest candidates are:
[0m  lmul!([91m::Union{Adjoint{T, var"#s814"} where var"#s814"<:(LowerTriangular{T, var"#s813"} where var"#s813"<:(Union{SparseArrays.AbstractSparseMatrixCSC{T, Ti}, SubArray{T, 2, var"#s814", Tuple{Base.Slice{Base.OneTo{Int64}}, I}, L} where {I<:AbstractUnitRange, var"#s814"<:SparseArrays.AbstractSparseMatrixCSC{T, Ti}, L}} where Ti)), Adjoint{T, var"#s814"} where var"#s814"<:(UpperTriangular{T, var"#s813"} where var"#s813"<:(Union{SparseArrays.AbstractSparseMatrixCSC{T, Ti}, SubArray{T, 2, var"#s814", Tuple{Base.Slice{Base.OneTo{Int64}}, I}, L} where {I<:AbstractUnitRange, var"#s814"<:SparseArrays.AbstractSparseMatrixCSC{T, Ti}, L}} where Ti)), Adjoint{T, var"#s812"} where var"#s812"<:(UnitUpperTriangular{T, var"#s811"} where var"#s811"<:(Union{SparseArrays.AbstractSparseMatrixCSC{T, Ti}, SubArray{T, 2, var"#s814", Tuple{Base.Slice{Base.OneTo{Int64}}, I}, L} where {I<:

##### Calculate the energy per spin

Dividing both sides of the Hamiltonian yields the Hamiltonian in units of J and gives us a tuneable parameter in front of the Transverse Field (h/J)

$\frac{H}{J} = -\sum_{i=1}^{N-1} \sigma_{i}^{z}\sigma_{i+1}^{z} - \sigma_{N}\sigma_{1} - \frac{h}{J}\sum_{i = 1}^{N}\sigma_{i}^{x}$


In [ ]:
N  = 14
J  = 1
h_J = 0 #H' is h/J
H = Hamiltonian(N,J,h_J)
Evals = eigvals(H)

TotalE = sum(Evals)
E_per_spin = TotalE/n_
println(Evals)
println(TotalE)

From previous assignment (reading course, Exact diag part 2)

In [ ]:
using LinearAlgebra

N_many = 14
Ns = zeros(N_many)
eigmins = zeros(N_many)
J_ = 1
B_ = 0.5

for i in (1:N_many)
    Ni = i
    Ns[i] = Ni
    Hi = GeneralHamiltonian(Ni,J_,B_)
    eigmins[i] = eigmin(Hi)
end


In [ ]:
using Plots

eigmin_perspin = zeros(N_many)

for i in (1:N_many)
    eigmin_perspin[i] = eigmins[i]/Ns[i]
end

plot(Ns,eigmin_perspin,label = "min eigenvalue per spin")